## 特征词提取降维

In [1]:
#统计词频和文档数,考虑词频来计算CHI
#wroddict={tf:出现次数；total:该类总单词数；idf:出现的文档数目}
import json
import os
from collections import Counter
import time
def count_words(cut_path):
    cate=os.listdir(cut_path)
    wordDict={'total':{}}
    for i,category in enumerate(cate):
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()), '>', '=' * 40 + '[' + category + ']' + '=' * 40)
        wordInfo={'tf':0,'idf':1}
        wordsLen=0
        file_path=cut_path+category+'/'
        file_list=os.listdir(file_path)
        for j,file_name in enumerate(file_list):
            full_path=file_path+file_name
            with open(full_path,"r",encoding='utf-8') as f:
                content=f.read()
            words=content.split()
            wordsLen+=len(words)
            wordCounter=Counter(words)
            for index,wordTuple in enumerate(wordCounter.most_common(len(wordCounter))):
                (word,count)=wordTuple
                wordDict.setdefault(word,{})
                wordDict[word].setdefault(category,wordInfo.copy())
                wordDict[word][category]['tf']+=count
                wordDict[word][category]['idf']+=1
        wordDict['total'][category]=wordDict.setdefault(category,wordsLen)
    wordDict=dict(wordDict)
    fname='C:/lyr/DM/feature_reduction/wordDict.json'
    with open(fname,'w') as fp:
        json.dump(wordDict,fp)

In [2]:
p='C:/lyr/DM/train_cut/'
count_words(p)

2019-01-17 09:03:53 > ========================================[Auto]========================================
2019-01-17 09:05:08 > ========================================[business]========================================
2019-01-17 09:06:28 > ========================================[edu]========================================
2019-01-17 09:07:19 > ========================================[ent]========================================
2019-01-17 09:08:00 > ========================================[healthy]========================================
2019-01-17 09:08:43 > ========================================[mil]========================================
2019-01-17 09:09:09 > ========================================[policy]========================================
2019-01-17 09:09:43 > ========================================[sports]========================================
2019-01-17 09:10:21 > ========================================[tech]========================================
2019-01-17 

In [3]:
#计算每类有多少文档
docLen={}
def getDoclen(cut_path):
    cate=os.listdir(cut_path)
    alldocLen=0
    for i,category in enumerate(cate):
        file_path=cut_path+category+'/'
        length=len(os.listdir(file_path))
        alldocLen+=length
        docLen.setdefault(category,length)
    docLen.setdefault('All',alldocLen)
    fname='C:/lyr/DM/feature_reduction/docLen.json'
    with open(fname,'w') as fp:
        json.dump(docLen,fp)
getDoclen('C:/lyr/DM/train_cut/')

## CHI

In [4]:
#计算每个类的每个词的卡方，我的结构调一下会不会好些。算了都差不多。
path='C:/lyr/DM/feature_reduction/wordDict.json'
with open(path, "r",encoding='utf-8') as fp:
    ddict=json.load(fp)
path2='C:/lyr/DM/feature_reduction/docLen.json'
with open(path2,"r",encoding='utf-8') as fp:
    docLen=json.load(fp)

In [5]:
from math import log
weightDict={}
featureDict={'chi':0,'tfidf':'0'}
#weightDict={chi: ,tfidf:}都一起算了吧
def gen_weightDict():
    for (word,value) in ddict.items():
        if word!='total':
            weightDict.setdefault(word,{})
            totalCount=0#A+B该词出现的总次数
            try:
                for (cate,times) in value.items():
                    totalCount+=times['tf']
            except Exception as ex:
                print(word,value)
            try: 
                for (cate,times) in value.items():
                    weightDict[word].setdefault(cate,featureDict.copy())
                    #乘除100，防止太大太小！？
                    weightDict[word][cate]['tfidf']=(times['tf']/ddict['total'][cate])*100*log((docLen[cate]+1)/times['idf'])
                    #times['idf']#相当于a
                    not_in_Class=totalCount-times['idf']#相当于B
                    not_has_word=docLen[cate]-times['idf']#相当于c
                    d=docLen['All']-totalCount-not_has_word
                    chi=pow((times['idf']*d-not_in_Class*not_has_word),2)/(100*totalCount*(d+not_has_word))
                    weightDict[word][cate]['chi']=chi
            except Exception as ex:
                pass
    fname='C:/lyr/DM/feature_reduction/weigthDict.json'
    with open(fname,'w') as fp:
        json.dump(weightDict,fp)

In [6]:
gen_weightDict()

Auto 6233150
business 11313717
edu 8543850
ent 4755005
healthy 5189905
mil 2927406
policy 3194016
sports 3939763
tech 4973478
tourism 4864357
women 14737365


In [7]:
len(weightDict)

524560

In [8]:
weightDict

{'汽车': {'Auto': {'chi': 2992096.472400395, 'tfidf': 1.7554597692245393},
  'business': {'chi': 25260428.92542144, 'tfidf': 0.40781430050117146},
  'edu': {'chi': 14504687.814203829, 'tfidf': 0.07929478019990803},
  'ent': {'chi': 27455807.182350222, 'tfidf': 0.08503620948296547},
  'healthy': {'chi': 5936957.29832584, 'tfidf': 0.035237146581440136},
  'mil': {'chi': 9101826.020402469, 'tfidf': 0.10448175694708058},
  'policy': {'chi': 4445236.746022159, 'tfidf': 0.24326672457057977},
  'sports': {'chi': 19813513.554181255, 'tfidf': 0.052975927387373646},
  'tech': {'chi': 5230900.760118038, 'tfidf': 0.6830821409441218},
  'tourism': {'chi': 7773376.833283777, 'tfidf': 0.1803571370467725},
  'women': {'chi': 9643527.446852058, 'tfidf': 0.014611096574247966}},
 '吉利': {'Auto': {'chi': 597261.2856328146, 'tfidf': 0.3627572960517788},
  'business': {'chi': 661814.3060297887, 'tfidf': 0.02098139444284927},
  'edu': {'chi': 313564.63316060597, 'tfidf': 0.014727757762379659},
  'ent': {'chi': 

In [10]:
len(ddict)

524561

In [11]:
#要换一下组织方式，不然不会排序。。。
bothResult=[]
def reGen():
    tempDict={}
    for word,value in weightDict.items():
        for cate,weight in value.items():
            tempDict.setdefault(cate,{})
            tempDict[cate].setdefault(word,{'chi':0,'tfidf':0,'both':0})
            tempDict[cate][word]['chi']=weight['chi']
            tempDict[cate][word]['tfidf']=weight['tfidf']
            tempDict[cate][word]['both']=weight['chi']*weight['tfidf']
    fname='C:/lyr/DM/feature_reduction/new_weigthDict.json'
    with open(fname,'w') as fp:
        json.dump(tempDict,fp)

In [12]:
reGen()

In [14]:
path3='C:/lyr/DM/feature_reduction/new_weigthDict.json'
with open(path3, "r",encoding='utf-8') as fp:
    new_weigthDict=json.load(fp)
new_weigthDict

{'Auto': {'汽车': {'chi': 2992096.472400395,
   'tfidf': 1.7554597692245393,
   'both': 5252504.982937556},
  '吉利': {'chi': 597261.2856328146,
   'tfidf': 0.3627572960517788,
   'both': 216660.88901256895},
  '瑞典': {'chi': 95005.34354099256,
   'tfidf': 0.042754244649410106,
   'both': 4061.88170075285},
  '企业': {'chi': 6755871.473224049,
   'tfidf': 0.8497671729802534,
   'both': 5740917.80281954},
  '中国': {'chi': 377813405.614624,
   'tfidf': 1.3074539906021978,
   'both': 493973644.873847},
  '巨头': {'chi': 154460.09952639454,
   'tfidf': 0.0914648417255655,
   'both': 14127.66855609677},
  '部门': {'chi': 864471.7833768289,
   'tfidf': 0.30602978879771303,
   'both': 264554.11728839326},
  '旗下': {'chi': 2824438.681356047,
   'tfidf': 0.27933173631471,
   'both': 788955.3609776144},
  '资金': {'chi': 2746275.7826870587,
   'tfidf': 0.19267547382713268,
   'both': 529139.9876892087},
  '优先股': {'chi': 38.53725219854709,
   'tfidf': 0.002484225789795045,
   'both': 0.09573523577946647},
  '上市

In [15]:
resultDict={'chi':{},'tfidf':{},'both':{}}#存3种结果
maxCount=8000
def gen_featureDict(maxCount):
    for cate,words in new_weigthDict.items():
        print(cate)
        chiMix=dict(sorted(words.items(),key=lambda x:x[1]['chi'],reverse=True)[:maxCount])
        tdidfMix=dict(sorted(words.items(),key=lambda x:x[1]['tfidf'],reverse=True)[:maxCount])
        bothMix=dict(sorted(words.items(),key=lambda x:x[1]['both'],reverse=True)[:maxCount])
        resultDict['chi'].setdefault(cate,{})
        resultDict['tfidf'].setdefault(cate,{})
        resultDict['both'].setdefault(cate,{})
        chiSum,tfidfSum,bothSum=0,0,0
        #我想把里面混的别的数据分出去
        for word,weight in chiMix.items():
            chiSum+=weight['chi'] #虽然能放下但就是觉得好大
            resultDict['chi'][cate].setdefault(word,weight['chi'])
        for word,weight in tdidfMix.items():
            tfidfSum+=weight['tfidf']
            resultDict['tfidf'][cate].setdefault(word,weight['tfidf'])
        for word,weight in bothMix.items():
            bothSum+=weight['both']
            resultDict['both'][cate].setdefault(word,weight['both'])
        #结果到0-1之间，不然chi乘起来就gg了
        for word,weight in chiMix.items():
            resultDict['chi'][cate][word]/=chiSum
        for word,weight in tdidfMix.items():
            resultDict['tfidf'][cate][word]/=tfidfSum
        for word,weight in bothMix.items():
            resultDict['both'][cate][word]/=bothSum
gen_featureDict(maxCount)

Auto
business
edu
ent
healthy
mil
policy
sports
tech
tourism
women


In [16]:
resultDict['tfidf']

{'Auto': {'汽车': 0.0057452495931300755,
  '新能源': 0.005211055685899437,
  '中国': 0.004279021165413425,
  '市场': 0.00406906803872367,
  '品牌': 0.00366843733693281,
  '销量': 0.0032405272899146084,
  '车辆': 0.003053442829920825,
  '车型': 0.003052368217796998,
  '公司': 0.003040454660237753,
  '企业': 0.0027811087388101953,
  '奥迪': 0.002700308165444735,
  '记者': 0.002604824130019488,
  '发动机': 0.0025604299518837757,
  '二手车': 0.0025548020744587696,
  '产品': 0.002554756564396695,
  '北京': 0.0025129181019368668,
  '消费者': 0.002448028480718534,
  '东风': 0.002401394079076272,
  '出租车': 0.0023837540665698153,
  '系统': 0.0023766176913647017,
  '技术': 0.0023655548753458685,
  '车': 0.0023587265471262816,
  '店': 0.0023517261425382912,
  '经销商': 0.0023204025624882782,
  '大众': 0.0022920585217361543,
  '车主': 0.002290063151940978,
  '新车': 0.0021849644032840678,
  '丰田': 0.0020916887377995697,
  '人': 0.0020706522943585064,
  '价格': 0.0020258708329522752,
  '专车': 0.0020235494463933285,
  '4S店': 0.0020196553318086014,
  '动力': 0.0